In [ ]:
%matplotlib inline
import matplotlib
import time
from pylab import *
matplotlib.rcParams['figure.figsize'] = (8, 8)
import matplotlib.gridspec as gridspec
from IPython import display
import numpy as np

def weights_line(w: np.ndarray):
    "Return x1,y1,x2,y2 line for given perceptron weights."
    
    y1, y2 = 0, 1
    x1 = (-y1 * w[1] - w[2]) / w[0]
    x2 = (-y2 * w[1] - w[2]) / w[0]
    return x1, y1, x2, y2

def loss(w: np.ndarray, points: np.ndarray, classes: np.ndarray):
    "Get overall error for points and given perceptron weights."
    
    out = np.dot(w, points)
    error1 = out[np.logical_and(out < 0, classes == 1)].sum()
    error2 = out[np.logical_and(out > 0, classes == 0)].sum()
    return np.abs(error1 + error2)

gs = gridspec.GridSpec(5, 1)               # Init plots grid with height ration 5:1.
ax = plt.subplot(gs[0:4, 0])               # Init points subplot.
ax.set_xlabel('x1'); ax.set_ylabel('x2')   # Set points plot x and y labels.
ax2 = plt.subplot(gs[4, 0])                # Init loss subplot.
ax2.set_xlabel('Iteration')                # Set loss plot y label.
ax2.set_ylabel('Loss')                     # Set loss plot y label.

w_ideal = np.array([1, -1, -0.2])          # Define and draw ideal separating line.
x1, y1, x2, y2 = weights_line(w_ideal)     # Get end-to-end line coordinates.
ax.plot([x1, x2], [y1, y2], 'c-');         # Draw line.
ax.set_xlim(0, 1); ax.set_ylim(0, 1)       # Set points plot axes limits.
ax.set_aspect('equal')                     # Set points plot axes equal.

n_samples = 50                             # Set points count.
points = np.random.rand(3, n_samples)      # Generate points.
classes = np.zeros(n_samples)              # Init classes array.
points[2,:] = 1                            # Set virtual x3 value of each point to 1 (input for bias weight).
classes[np.dot(w_ideal, points) > 0] = 1   # Set positive classified points class to 1.
ax.plot(points[0, classes == 1], points[1, classes == 1], 'ro');  # Draw positive points as red.
ax.plot(points[0, classes == 0], points[1, classes == 0], 'bo');  # Draw negative points as blue.

w = np.random.rand(3)                      # Init perceptron weights to random values.
i_iter = 0                                 # Init iteration index.
line, = ax.plot([], [])                    # Init perceptron plot line data.
ls_data, = ax2.plot([], [], 'b-')          # Init loss plot data.
lss, itr = [], []                          # Init lists for loss plot data.
while True:                                # Train perceptron with loop.
    i_point = i_iter % points.shape[1]     # Get next/first point index.
    point = points[:, i_point];            # Get point to test and correct perceptron weights.
    p = 1 if np.dot(w, point) > 0 else 0   # Get prediction for current point (1 for positive, 0 for negative).
    c = classes[i_point] - p               # Get correction: 0 if class equals prediction.
    w += c * point                         # Correct perceptron weights.                            
    lss.append(loss(w, points, classes))   # Compute perceptron overall loss on points.
    itr.append(i_iter)                     # Update iteration list.
    i_iter += 1                            # Increment iteration index.
    if c != 0:                             # Update plot if perceptron changed.
        x1, y1, x2, y2 = weights_line(w)   # Get perceptron line end-to-end coordinates.
        line.set_data([x1, x2], [y1, y2]); # Update perceptron visualisation line data.
        ax.set_title('Loss: %.2f, iteration: %d' % (lss[-1], i_iter)) # Display current loss and iteration number.
        ls_data.set_data(itr, lss)         # Update loss plot.
        ax2.set_xlim(-10, max(100, max(itr))) # Scale loss plot - x.
        ax2.set_ylim(0, max(10, max(lss))) # Scale loss plot - y.
        display.clear_output(wait=True)    # Redraw...
        display.display(gcf())             # ...plot.
        time.sleep(0.1)                    # Wait a moment to see a plot.
    if lss[-1] == 0:                           # End if...
        break                              # ...perceptron is trained.